In [0]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import load_model
import vis, keras


# Authenticate and create the PyDrive client.  #用來驗證
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
id1 = '1zeiBnux7KgCTfB6nwAUB8p3as9FJayxF'
id2 = '1NqfACLqTByXSNPunB_zH0JoVTtSjFVE_'
id3 = '16l6CeS_frdb1HBw2YuGbXSczN4eQlpdk'

#這邊要放檔案名稱
downloaded1 = drive.CreateFile({'id': id1})
downloaded2 = drive.CreateFile({'id': id2})
downloaded3 = drive.CreateFile({'id': id3})

downloaded1.GetContentFile('train.csv')
downloaded2.GetContentFile('test.csv')
downloaded3.GetContentFile('sample_submission.csv')

df = pd.read_csv('train.csv')
df_testing = pd.read_csv('test.csv')

In [0]:
#把data做validation
from sklearn.model_selection import train_test_split
X = df.drop('label', axis = 1)
Y = df['label']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 3)

In [0]:
#轉成可以用的data模式
def shape_CNN(data):
  temp = []
  for i in range(data.shape[0]):
    temp.append(np.array(data.iloc[i,0].split(" "), dtype = 'float').reshape(48, 48, 1))  #第一個1代表色階
  return np.array(temp) / 255  #做個標準化

In [0]:
from keras.utils import np_utils

X_train = shape_CNN(X_train)
X_test = shape_CNN(X_test)
Y_train = np_utils.to_categorical(Y_train, 7)
Y_test = np_utils.to_categorical(Y_test, 7)

In [0]:
import keras
from keras.layers import DepthwiseConv2D, Conv2D, AveragePooling2D, BatchNormalization, Input, MaxPooling2D
from keras.layers import Dense, Activation, Flatten, Dropout, Input
from keras.layers import LeakyReLU
from keras.models import Sequential, Model

Mobilenet V1

In [0]:
def identity_block(model, filters, Con = True):
  kernel_size = (3,3)
  pool_size = (2,2)
  kernel_initializer = keras.initializers.glorot_normal()
  
  if Con == True:
    #先針對每一個channel
    x = DepthwiseConv2D(kernel_size = kernel_size, padding = 'same')(model) 
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    
    #再用1x1把用完Depthwise的東西作轉換, 最後再做pooling
    x = Conv2D(filters = filters, kernel_size = (1,1), padding = 'same', kernel_initializer = kernel_initializer)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    x = MaxPooling2D(pool_size = pool_size, padding = 'same')(x)
    return x
  else:
    x = Dense(units = filters, kernel_initializer = kernel_initializer)(model)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    return x

In [0]:
input_img = Input(shape = (48,48,1))

#第一層先做一般的CNN
Mobilenet = Conv2D(filters = 128, kernel_size = (3,3), padding = 'same', kernel_initializer = keras.initializers.glorot_normal())(input_img)
Mobilenet = BatchNormalization()(Mobilenet)
Mobilenet = LeakyReLU()(Mobilenet)
Mobilenet = MaxPooling2D(pool_size = (2,2), padding = 'same')(Mobilenet)

Mobilenet = identity_block(Mobilenet, 128)
Mobilenet = identity_block(Mobilenet, 256)
Mobilenet = identity_block(Mobilenet, 256)

Mobilenet = Flatten()(Mobilenet)
Mobilenet = identity_block(Mobilenet, 64, Con = False)
Mobilenet = identity_block(Mobilenet, 64, Con = False)

Mobilenet = Dense(units = 7, activation='softmax')(Mobilenet)

Mobilenet = Model(input_img, Mobilenet)  
Mobilenet.compile(optimizer='adamax', loss='categorical_crossentropy', metrics = ['accuracy'])

In [0]:
#Image generator
from keras.preprocessing.image import ImageDataGenerator
img_generator = ImageDataGenerator(rotation_range = 20, width_shift_range = 0.2, height_shift_range = 0.2, zoom_range = 0.3)

In [25]:
#callback函數
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

lr_adjust = ReduceLROnPlateau(monitor='val_acc', factor=0.1, patience = 4, verbose = 1, mode='auto', epsilon=0.0001, cooldown=0, min_lr=0)
earlystopping = EarlyStopping(monitor='val_acc', patience = 4, verbose = 1, mode='auto')

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:1335: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


In [47]:
epochs = 1
batch_size = 64

Mobilenet.fit_generator(img_generator.flow(X_train, Y_train, batch_size = batch_size), 
                        steps_per_epoch=len(X_train)/12, 
                        validation_data=(X_test, Y_test), 
                        epochs = epochs,
                        callbacks = [lr_adjust, earlystopping])

Epoch 1/1
1675/1674 [==============================] - 117s 70ms/step - loss: 0.7393 - acc: 0.7256 - val_loss: 1.0766 - val_acc: 0.6362


最後輸出

In [0]:
df_testing = df_testing.drop('id', axis = 1)
df_testing = shape_CNN(df_testing)
result_final = Mobilenet.predict(df_testing)
sample_submission = pd.read_csv('sample_submission.csv', header = 0,encoding = 'unicode_escape')

In [0]:
for i in range(result_final.shape[0]):
  sample_submission.iloc[i,1] = int(np.argmax(result_final[i]))

In [0]:
from google.colab import files
sample_submission.to_csv('submit.csv', index = False)
files.download('submit.csv')

In [0]:
Mobilenet.save('my_model.h5')
files.download('my_model.h5')